In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/

/content/drive/MyDrive


## Import important libraries

In [8]:
import spacy
import re
import pandas as pd

## Load Predictions dataset

In [23]:
preds = pd.read_csv('Speech2TextPredictions.csv')

In [26]:
preds.to_csv('Speech2TextPredictions.csv')

### Converting words quantities into numeric

In [27]:
urdu_counting = {'0.25': 'ایک چوتھائی', '0.5': 'آدھا', '½-1': 'ڈیڑھ','1½': 'ڈیڑھ', '½-2': 'ڈھائی',
 '2½': 'ڈھائی', '½': 'آدھا', '1': 'ایک', '2': 'دو', '3': 'تین', '4': 'چار', '5': 'پانچ', '6': 'چھ',
 '7': 'سات', '8': 'آٹھ', '9': 'نو', '10': 'دس', '11': 'گیارہ', '12': 'بارہ', '13': 'تیرہ', '14': 'چودہ',
 '15': 'پندرہ', '16': 'سولہ', '17': 'سترہ', '18': 'اٹھارہ', '19': 'انیس', '20': 'بیس', '21': 'اکیس', '22': 'بائیس',
 '23': 'تئیس', '24': 'چوبیس', '25': 'پچیس', '26': 'چھببیس', '27': 'ستائیس', '28': 'اٹھائیس', '29': 'انتیس', '30': 'تیس',
 '31': 'اکتیس', '32': 'بتیس', '33': 'تینتیس', '34': 'چونتیس', '35': 'پینتیس', '36': 'چھتیس', '37': 'سینتیس', '38': 'اڑتیس',
 '39': 'انتالیس', '40': 'چالیس', '41': 'اکتالیس', '42': 'بیالیس', '43': 'تینتالیس', '44': 'چوالیس', '45': 'پینتالیس', '46': 'چھیالیس',
 '47': 'سینتالیس', '48': 'اڑتالیس', '49': 'انچاس', '50': 'پچاس', '51': 'اکیاون', '52': 'باون', '53': 'ترپن', '54': 'چون', 
 '55': 'پچپن', '56': 'چھپن', '57': 'ستاون', '58': 'اٹھاون', '59': 'انسٹھ', '60': 'ساٹھ', '61': 'اکسٹھ', '62': 'باسٹھ',
 '63': 'ترسٹھ', '64': 'چوسٹھ', '65': 'پینسٹھ', '66': 'چھیاسٹھ', '67': 'سڑسٹھ', '68': 'اٹھسٹھ', '69': 'انہتر', '70': 'ستر',
 '71': 'اکہتر', '72': 'بہتر', '73': 'تہتر', '74': 'چوہتر', '75': 'پچہتر', '76': 'چھہتر', '77': 'ستتر', '78': 'اٹھہتر',
 '79': 'اناسی', '80': 'اسی', '81': 'اکیاسی', '82': 'بیاسی', '83': 'تراسی', '84': 'چوراسی', '85': 'پچاسی', '86': 'چھیاسی',
 '87': 'ستاسی', '88': 'اٹھاسی', '89': 'نواسی', '90': 'نوے', '91': 'اکانوے', '92': 'بانوے', '93': 'ترانوے', '94': 'چورانوے',
 '95': 'پچانوے', '96': 'چھیانوے', '97': 'ستانوے', '98': 'اٹھانوے', '99': 'ننانوے', '100': 'سو', '150': 'ایک سو پچاس', '200': 'دو سو',
 '250': 'دو سو پچاس', '300': 'تین سو', '350': 'تین سو پچاس', '400': 'چار سو',
 '450': 'چار سو پچاس', '500': 'پانچ سو'}

In [ ]:
nlp = spacy.load("Urdu_RecipeNER_Model")

In [31]:
urdu_numbers = {v: k for k, v in urdu_counting.items()}
predictions = preds.replace(urdu_numbers, regex = True)

In [32]:
predictions.head()

,Unnamed: 0,Recipe Name,Original Text,Whisper Predicted Text,Wav2Vec2 Predicted Text
0,0,بلوچی گوشت,بلوچی گوشت بنانے کے اجزاء ، بون لیس مٹن ½ کلو،...,بلوچی گوشڈر آدہ چائے کا چمچہ شملا مرچ 1 ادد ک...,بلوچی گوشت بنانے کے سا مول لیس مٹ دھکیل ادرک ل...
1,1,فروٹی اسموتھی,فروٹی اسموتھی بنانے کے اجزاء ، آڑو 1 کپ، درمی...,فروٹی سمودی بنانے کے اتزا اڑو 1 کپ ترمیانے گیل...,فروٹی اس مودی بنانے کے عجز اڑو 1 کپ درمیانے کی...
2,2,چلی چلی جھینگے,چلی چلی جھینگے بنانے کے اجزاء جھینگے ½ کلو، نا...,چلی چلی جھنگے بنانے کے اجزا جھنگے ½ کلو ناریل ...,چلی چلی جین گے بنانے کے جزا جیگ دھکیل ناریل قو...
3,3,کرسپ اسپنچ وِد سیلیڈ,کرسپ اسپنچ و د سیلیڈ بنانے کے اجزاء ، چکن بریس...,کرسپ سپنچ ویٹ سالٹ بنانے کے اجزاء چکن بریسٹ 2 ...,کرسپسپنچوت سالٹ بنانے کے از چکن برس 2 عدد کھیر...
4,4,کریم آلو,کریم آلو بنانے کے اجزاء ، آلو 1 کلو، ہری پیاز ...,کریم آلو بنانے کے عدزہ آلو ½ کلو ہری پیاز 2 عد...,کریم آلو بنانے کے عصا آلو ادا کیلوں ہری پیاز 2...


In [34]:
predictions['Original Text'][1]

' فروٹی اسموتھی بنانے کے اجزاء ، آڑو 1 کپ، درمیانی کیلے 2 عدد، دہی 1 کپ، برف 1 کپ، چینی 1 کھانے کا چمچہ'

In [36]:
tagged_entities = []
def tag_input(input_str):
  org_string = input_str 
  # Define the regex pattern
  pattern = r'(\S+)\s*(\d+(?:\.\d+)?)\s*(\S+)'
  # Extract the matches using the pattern
  matches = re.findall(pattern, input_str)
  decreased_length = 0
  # Create a list of tuples with the tagged entities
  for match in matches:
    ingredient = re.search(match[0].strip(), input_str)
    quantity = re.search(match[1].strip(), input_str)
    unit = re.search(match[2].strip(), input_str)
    text = input_str[ingredient.start(): ingredient.end()] + ' ' + input_str[quantity.start(): quantity.end()] + ' ' + input_str[unit.start(): unit.end()]
    annotations = {'entities': [(ingredient.start() + decreased_length, ingredient.end() + decreased_length, "Ingredient"), 
                                          (quantity.start() + decreased_length, quantity.end() + decreased_length, "Quantity"), 
                                          (unit.start() + decreased_length, unit.end() + decreased_length, "Unit")]}
    input_str = input_str[unit.end()+1:]
    decreased_length = unit.end()
    tagged_entities.append((text, annotations))


In [39]:
for i in predictions['Whisper Predicted Text']:
    tag_input(i) 

In [41]:
tagged_entities[:5]

[('پیسٹ 1 کھانے',
  {'entities': [(60, 64, 'Ingredient'),
    (65, 66, 'Quantity'),
    (67, 72, 'Unit')]}),
 ('مرچ 6 عدد،',
  {'entities': [(90, 93, 'Ingredient'),
    (94, 95, 'Quantity'),
    (96, 100, 'Unit')]}),
 ('مرچ 3 عدد،',
  {'entities': [(32, 35, 'Ingredient'),
    (36, 37, 'Quantity'),
    (38, 42, 'Unit')]}),
 ('زیرہ 1 چائے',
  {'entities': [(22, 26, 'Ingredient'),
    (27, 28, 'Quantity'),
    (29, 33, 'Unit')]}),
 ('دھنیا 1 /4',
  {'entities': [(33, 38, 'Ingredient'),
    (39, 40, 'Quantity'),
    (40, 42, 'Unit')]})]

In [43]:
# Function to evaluate the model on a test set
def evaluate_model(test_data):
    # Initialize counters
    total_true = 0
    total_predicted = 0
    total_correct = 0
    
    # Iterate over the test data
    for text, annotations in test_data:
        # Get the true entities
        true_entities = set([(ent[0], ent[1], ent[2]) for ent in annotations["entities"]])
        
        
        # Get the predicted entities
        doc = nlp(text)
        predicted_entities = set([(ent.start_char, ent.end_char ,ent.label_) for ent in doc.ents])
        
        # Update the counters
        total_true += len(true_entities)
        total_predicted += len(predicted_entities)
        total_correct += len(true_entities.intersection(predicted_entities))
    
    # Calculate precision, recall, and F1 score
    precision = total_correct / total_predicted if total_predicted > 0 else 0.0
    recall = total_correct / total_true if total_true > 0 else 0.0
    f1_score = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0.0
    
    # Print the evaluation metrics
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1 score: {f1_score:.2f}")

# Evaluate the model on the test data
evaluate_model(tagged_entities)

Precision: 0.01
Recall: 0.01
F1 score: 0.01
